In [1]:
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import pandas as pd
np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=4)

In [2]:
with open('Crimes_-_2001_to_present.csv') as infl:
    head = infl.readline().split(',')
    i=0
    for h in head:
        print i,' ',h
        i+=1

0   ID
1   Case Number
2   Date
3   Block
4   IUCR
5   Primary Type
6   Description
7   Location Description
8   Arrest
9   Domestic
10   Beat
11   District
12   Ward
13   Community Area
14   FBI Code
15   X Coordinate
16   Y Coordinate
17   Year
18   Updated On
19   Latitude
20   Longitude
21   Location



In [3]:
full_set = pd.read_csv('Crimes_-_2001_to_present.csv',sep=",",usecols=[2,5,8,9,13],quotechar='"',dtype='object')
full_set['Date'] =  [dt.datetime.strptime(d,'%m/%d/%Y %I:%M:%S %p') for d in full_set['Date']]
full_set['year'] = [d.year for d in full_set['Date']]

In [4]:
print full_set.shape
print full_set[:10]

(6208265, 6)
                 Date         Primary Type Arrest Domestic Community Area  \
0 2016-07-13 00:30:00  MOTOR VEHICLE THEFT  false    false             24   
1 2016-07-13 04:52:00      CRIMINAL DAMAGE  false     true             69   
2 2016-07-13 08:00:00      CRIMINAL DAMAGE  false    false             25   
3 2016-07-13 12:00:00                THEFT  false    false             66   
4 2016-07-13 13:27:00        OTHER OFFENSE  false     true             42   
5 2016-07-13 21:00:00      CRIMINAL DAMAGE  false    false             43   
6 2016-07-13 19:30:00                THEFT  false    false             15   
7 2016-07-13 10:00:00              BATTERY  false     true             22   
8 2016-07-13 13:00:00              ROBBERY  false    false             22   
9 2016-07-13 22:00:00      CRIMINAL DAMAGE  false    false             43   

   year  
0  2016  
1  2016  
2  2016  
3  2016  
4  2016  
5  2016  
6  2016  
7  2016  
8  2016  
9  2016  


In [5]:
type_agg = full_set.groupby(by=['Primary Type'])['Date'].count()

In [6]:
type_agg
plt.bar(range(0,35),type_agg)
plt.xticks(range(0,35),type_agg.index,rotation='vertical')
plt.show()

In [7]:
full_set['commNum'] = [int(c) for c in full_set['Community Area'].fillna(-1)]
plt.hist(full_set['commNum'],bins=range(-1,78))
plt.show()

In [8]:
keep_types=['ASSAULT','BATTERY','BURGLARY','CRIMINAL DAMAGE','CRIMINAL TRESPASS',\
                'DECEPTIVE PRACTICE','MOTOR VEHICLE THEFT','NARCOTICS','OTHER OFFENSE','ROBBERY','THEFT']

In [9]:
with open('Crime-2014-present.csv','w') as outf:
    outf.write("Date,Type,Arrest,Domestic,Community\n")
    j=0
    for i in range(full_set.shape[0]):
        if full_set['year'][i]>2013 and full_set['Primary Type'][i] in keep_types:
            if full_set['Primary Type'][i] in ['ASSAULT','BATTERY','ROBBERY']:
                ctype = 'Violent'
            elif full_set['Primary Type'][i] in ['CRIMINAL TRESPASS','DECEPTIVE PRACTICE','NARCOTICS','OTHER OFFENSE']:
                ctype = 'Other'
            else:
                ctype = 'Property'
            j+=1
            outs = str(full_set['Date'][i])+','+ctype+\
                ','+str(full_set['Arrest'][i])+','+str(full_set['Domestic'][i])+\
                ','+str(full_set['Community Area'][i])+'\n'
            outf.write(outs)
        else:
            continue
print j

725383


In [10]:
full_set = None

In [11]:
small_set = pd.read_csv('Crime-2014-present.csv',sep=",",quotechar='"',dtype='object')
small_set['Date'] =  [dt.datetime.strptime(d,'%Y-%m-%d %H:%M:%S') for d in small_set['Date']]
fd=dt.datetime.strptime('2013-12-31 00:00:01','%Y-%m-%d %H:%M:%S')
small_set['Day']=[(d-fd).days for d in small_set['Date']]

In [12]:
type_agg = small_set.groupby(by=['Type'])['Date'].count()
plt.bar(range(0,3),type_agg)
plt.xticks(range(0,3),type_agg.index)
plt.show()

In [13]:
small_set['commNum'] = [int(c) for c in small_set['Community'].fillna(-1)]
plt.hist(small_set['commNum'],bins=range(-1,78))
plt.show()

In [14]:
small_set.shape
small_set[:10]

,Date,Type,Arrest,Domestic,Community,Day,commNum
0,2016-07-13 00:30:00,Property,false,false,24,925,24
1,2016-07-13 04:52:00,Property,false,true,69,925,69
2,2016-07-13 08:00:00,Property,false,false,25,925,25
3,2016-07-13 12:00:00,Property,false,false,66,925,66
4,2016-07-13 13:27:00,Other,false,true,42,925,42
5,2016-07-13 21:00:00,Property,false,false,43,925,43
6,2016-07-13 19:30:00,Property,false,false,15,925,15
7,2016-07-13 10:00:00,Violent,false,true,22,925,22
8,2016-07-13 13:00:00,Violent,false,false,22,925,22
9,2016-07-13 22:00:00,Property,false,false,43,925,43


Here we aggregate the data over day, community, crime type, arrest and domestic - a sample of the output is shown below - and  saved to a csv. This is the data we will use to cluster over.

In [15]:
agg = small_set.groupby(by=['Day','Community','Type','Arrest','Domestic'])['Date'].count()
i=0
print agg[:10]
outs = "%d,%s,%s,%s,%s,%d\n"
with open('daily_hood_type.csv','w') as outf:
    outf.write("Day,Community,Type,Arrest,Domestic,Total\n")
    while i < len(agg):
        outf.write(outs%(agg.index[i][0],agg.index[i][1],agg.index[i][2],agg.index[i][3],agg.index[i][4],agg[agg.index[i]]))
        i+=1
print i

Day  Community  Type      Arrest  Domestic
0    11         Other     false   true        1
     14         Property  false   false       1
     16         Other     false   false       1
     17         Property  false   false       2
     22         Other     false   false       1
                Property  false   false       1
     24         Property  false   false       2
     25         Other     false   false       1
                Property  false   false       1
     26         Other     false   false       1
Name: Date, dtype: int64
308981
